In [ ]:
import pandas as pd
import Bio
from Bio import SeqIO
import itertools
import re
import numpy as np
import scipy as sp
from scipy import stats
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick 
from matplotlib.ticker import PercentFormatter

# FILE UPLOAD AND DICTIONARY GENERATION:

In [ ]:
# text file of target and nontarget names
targets = pd.read_csv('C:\\Users\\Alireza\\Documents\\MGY480\\Stoiber et al\\Individual RBPs\\Rbp1\\new_method\\posgenes.txt', header = None)
nontargets = pd.read_csv('C:\\Users\\Alireza\\Documents\\MGY480\\Stoiber et al\\Individual RBPs\\Rbp1\\new_method\\neggenes.txt', header = None)
RNAplfold_dir = 'C:\\Users\\Alireza\\Documents\\MGY480\\Stoiber et al\\Individual RBPs\\Rbp1\\new_method\\RNAplfold'

In [ ]:
# dictionary where keys are the fbtr and value is the list of sequence records
# if using fbtr:

fbtr2seqrecord = {}
for seqr in SeqIO.parse('C:\\Users\\Alireza\\Documents\\MGY480\\Hua_RIP\\brat_smg_targets20201210\\dmel-all-rna-r6.23.fasta','fasta'):
    fbtr = seqr.id
    fbtr2seqrecord[fbtr] = str(seqr.seq).replace('T','U') # str(seqr.seq) removes unwanted stuff (i.e. compare this line to the same thing but not using str); also, str allows you to use .replace

#filter dictionary such that only fbtr in target list are keys
target2seq = {key: fbtr2seqrecord[key] for key in targets[0]}
nontarget2seq = {key: fbtr2seqrecord[key] for key in nontargets[0]}

In [ ]:
# dictionary where keys are the fbtr and value is the list of sequence records
# if using fbgn (e.g. from stoiber set), then need longest transcript per gene:

gene2seqrecord = {}
for seqr in SeqIO.parse('C:\\Users\\Alireza\\Documents\\NSERC 2020\\Project 1\\Data Files\\dmel-all-transcript-r6.33.fasta','fasta'):
    gene = seqr.description.split(' ')[8][7:18]
    if gene in gene2seqrecord.keys():
        gene2seqrecord[gene].append(seqr)
    else:
        gene2seqrecord[gene] = [seqr]

# loop for all genes, sort the SeqRecord list by sequence length, longest first
# create new dic, fbgn2seq record where key is fbgn and value is seq of longest transcript
fbgn2seqrecord = {}
for gene, seqrlist in gene2seqrecord.items():
    seqrlist = sorted(seqrlist, key=lambda x:len(str(x.seq)), reverse=True)
    fbgn2seqrecord[gene] = str(seqrlist[0].seq).replace('T','U')

#filter dictionary such that only fbtr in target list are keys
target2seq = {key: fbgn2seqrecord[key] for key in targets[0]}
nontarget2seq = {key: fbgn2seqrecord[key] for key in nontargets[0]}

# FUNCTIONS:

In [ ]:
def consensus_to_motifs(consensus): # modification of #ATS fxn
    '''input = #ATS motif, output = all possible combinations of degenerate motifs'''

    seq_dic = {}
    seq_dic['A'] = 'A'
    seq_dic['C'] = 'C'
    seq_dic['U'] = 'U'
    seq_dic['G'] = 'G'
    seq_dic['H'] = 'ACU'
    seq_dic['W'] = 'AU'
    seq_dic['N'] = 'ACGU'
    seq_dic['M'] = 'AC'
    seq_dic['Y'] = 'UC'
    seq_dic['K'] = 'GU'
    seq_dic['B'] = 'CGU'
    seq_dic['D'] = 'AGU'
    seq_dic['R'] = 'GA'
    seq_dic['V'] = 'ACG'
    seq_dic['S'] = 'GC'
    
    motif = []
    for nt in consensus:
        motif.append(seq_dic[nt])
        
    motif_list = list(itertools.product(*motif))
    degen_motifs = []
    for comb in motif_list:
        degen_motifs.append(("".join(comb)))
        
    return degen_motifs

In [ ]:
# motif_list = consensus_to_motifs()
def transcript_removal(transcript2seq, motif_list):
    '''Remove transcripts that don't have any motifs from motif_list '''
    
    filtered = transcript2seq.copy()
    for fbtr, seq in list(filtered.items()): #.items puts key and value in brackets i.e. dict_items([(key1,val1), (key2,val2)]) --> allows unpacking of key, val pairs; need to use list because we are changing dic size
        n = 0 
        for motif in motif_list:
            if motif not in seq:
                n +=1 
        if n == len(motif_list): # this condition only occurs if none of the motifs in motif_list are present in a given transcript
            filtered.pop(fbtr)
    return filtered

In [ ]:
# Create dic where keys are targets and values are location of motifs 
def motif_occurences(transcript2seq, motif_list):
    '''return a new dic with key = fbtr and values = [seq,[occurences]]
    Occurence is stored as number of last nucleotide in motif'''
    
    copy_dic = transcript2seq.copy()
    for fbtr, seq in copy_dic.items():
        occurences = []
        for motif in motif_list:
            matches = re.finditer(motif, seq)
            matches_positions = [match.start() for match in matches]
            matches_positions = [m + len(motif)-1 for m in matches_positions]
            occurences += matches_positions
        copy_dic[fbtr] = [copy_dic[fbtr], occurences]
        
    return copy_dic

In [ ]:
# plfold_results = open(RNAplfold_dir + pos_gene + '_lunp', 'r')
def max_access(RNAplfold_direct, transcript2seq_occurences, mer_length):
    '''return a new dic, transcript2seq_occurence_max, with key = fbtr and values = ['seq', [motif w/ max accessibility], [max accessibility value]]
    RNAplfold_direct is the directory of RNAplfold results, transcript2seq_occurences is dic of key = fbtr and
    values = ['seq', [occurences]]'''
    
    copy_dic = transcript2seq_occurences.copy()
    for fbtr, values in copy_dic.items():
        plfold_results = open(RNAplfold_direct + '\\' + fbtr + '_lunp', 'r')
        plfold_df = pd.read_csv(plfold_results, sep = '\t', skiprows = 1)
        plfold_df = plfold_df.rename(columns={' #i$': 'nt', 'l=1': '1'})
        acc_list = []
        for occurence in values[1]: # values[1] is all motif occurences in transcript
            acc_list.append(plfold_df[str(mer_length)].iloc[occurence - 1]) # df["column"].iloc[row]; occurence - 1 = df index
        copy_dic[fbtr] = [copy_dic[fbtr][0],copy_dic[fbtr][1], acc_list]
    
    final = {}
    for fbtr, values in copy_dic.items():
        val = np.argmax(copy_dic[fbtr][2]) #argmin and argmax give index of min and max value, respectively
        final[fbtr] = [copy_dic[fbtr][0], copy_dic[fbtr][1][val], copy_dic[fbtr][2][val]]
        
    # calculation of average accessibility of max values
    total_acc = 0 
    num_transcripts = 0
    for fbtr, values in final.items():
        if not np.isnan(final[fbtr][2]):
            total_acc += final[fbtr][2]
            num_transcripts += 1
    avg_acc = (total_acc/num_transcripts)
    print ('The average accessibility of the most accessible motifs is: ' + str(avg_acc))
    
    # calculation of median accessibility of max values
    final_df = pd.DataFrame(final).T
    final_df_nona = final_df.dropna(axis = 0, subset=[2])
    print ('The median accessibility of the most accessible motifs is: ' + str(final_df_nona[2].median()))
        
    return final

def motif_access_wilcoxon(target2seq_occurence_max, nontarget2seq_occurence_max):
    '''Wilcoxon ranked sum test for comparison of motifs with maximum accessibility b/w targets and nontargets'''
    
    num_significant = 0
    num_non_significant = 0
    p_value_list = []
    target_df = pd.DataFrame(target2seq_occurence_max).T
    nontarget_df = pd.DataFrame(nontarget2seq_occurence_max).T
    
    # for _ in range(num_replicates):
        
    target_nona = target_df.dropna(axis=0, subset=[2])
    nontarget_nona = nontarget_df.dropna(axis=0, subset=[2])
        # nontarget_nona_sample = nontarget_nona.sample(n = len(target_nona))
        
    stat, p_value = sp.stats.mannwhitneyu(target_nona[2], nontarget_nona[2])
    p_value_list.append(p_value)
        
    if p_value < 0.05:
        num_significant +=1
    else:
        num_non_significant +=1 
            
    print ('Number of significant runs = ' + str(num_significant))
    print ('Number of non-significant runs =  ' + str(num_non_significant))
    print (sum(p_value_list) / float(len(p_value_list)))
    
    return target_df, nontarget_df # df: rows = fbtr, columns = [seq, max_motif, max_motif_value]

    
def motif_access_distribution(df, bin_increment, bin_max, percentage_tick, max_tick, RBP_name):
    #df returned in motif_access_wilcoxon
    #bin_increment is the increments on the y-axis, with bin_max being the max bin --> Judgement call based on data
    #percentage_ticks is the increment on x-axis inserted as a float (e.g. 2% increments = 0.02)
    # max_tick is the max y value --> judgement call; run with 1 first and then decide
    
    #create bin for histogram
    n = 0
    bin_list = [n]
    while n < bin_max: 
        n += bin_increment
        bin_list.append(n)
    
    no_na = df.dropna()
    data = no_na[2]
    
    fig, ax = plt.subplots()
    y_vals, x_vals, e_ = ax.hist(data, bin_list , edgecolor='black')
    # y_max = round((max(y_vals) / len(data)) + 0.05, 2)
    y_max = max_tick + 0.05
    ax.set_yticks(ticks=np.arange(0.0, y_max * len(data), percentage_tick * len(data)))
    ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
    ax.set_xlabel('accessibility')
    ax.set_title(RBP_name)
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=len(data)))
    
def seq_excerpt(transcript2seq_occurence_max, out_dir):
    '''create new file with sequence of motif and flanking region for each transcript in dic'''

    fname = out_dir
    fw = open(fname,'w')

    for fbtr, values in transcript2seq_occurence_max.items():
        val1 = transcript2seq_occurence_max[fbtr][1] - 1 - 25
        val2 = transcript2seq_occurence_max[fbtr][1] - 1 + 15

        if val1 > 0: 
            fw.write('>'+fbtr+'\n')
            fw.write(transcript2seq_occurence_max[fbtr][0][val1:val2 + 1] + '\n')
        else:
            fw.write('>'+fbtr+'\n')
            fw.write(transcript2seq_occurence_max[fbtr][0][0: val2 + 1] + '\n')


    fw.close()

In [ ]:
def flanking_access(transcript2seq_occurence_max, RNAplfold_direct, flank_length, mer_length):
    '''Return accessibility of region flanking motif w/ max accessibility
    transcrip2seq_occurence_max is key = fbtr and values = ['seq', [occurence of motif w/ max access], [max access value]]'''
    
    five_prime_acc_list = []
    three_prime_acc_list = []
    for fbtr, values in transcript2seq_occurence_max.items():
        plfold_results = open(RNAplfold_direct + '\\' + fbtr + '_lunp', 'r')
        plfold_df = pd.read_csv(plfold_results, sep = '\t', skiprows = 1)
        plfold_df = plfold_df.rename(columns={' #i$': 'nt', 'l=1': '1'})
        
        five_row =  transcript2seq_occurence_max[fbtr][1] - 1 - mer_length
        if five_row > 0:
            if not np.isnan(plfold_df[str(flank_length)].iloc[five_row]):
                five_prime_acc_list.append(plfold_df[str(flank_length)].iloc[five_row])
        
        three_row = transcript2seq_occurence_max[fbtr][1] - 1 + flank_length
        if three_row < len(transcript2seq_occurence_max[fbtr][0]):
            if not np.isnan(plfold_df[str(flank_length)].iloc[three_row]):
                three_prime_acc_list.append(plfold_df[str(flank_length)].iloc[three_row])
    
    #avg_five = np.mean(five_prime_acc_list)
    #avg_three = np.mean(three_prime_acc_list)
    combined = five_prime_acc_list + three_prime_acc_list
    avg_flank_acc = np.mean(combined)
    median_flank_acc = np.median(combined)
    
    # return avg_five, avg_three
    print('median access = ' + str(median_flank_acc))
    return avg_flank_acc

def flank_access_wilcoxon(target2seq_occurence_max, nontarget2seq_occurence_max, RNAplfold_direct, flank_length, mer_length):
    
    target2maxflank = {} # new dic to be populated; key = fbgtr and value = [max_flank_access]
    nontarget2maxflank = {}
    
    for fbtr, values in target2seq_occurence_max.items():
        plfold_results = open(RNAplfold_direct + '\\' + fbtr + '_lunp', 'r')
        plfold_df = pd.read_csv(plfold_results, sep = '\t', skiprows = 1)
        plfold_df = plfold_df.rename(columns={' #i$': 'nt', 'l=1': '1'})
        
        five_row =  target2seq_occurence_max[fbtr][1] - 1 - mer_length
        if five_row > 0:
            if not np.isnan(plfold_df[str(flank_length)].iloc[five_row]):
                five_prime = plfold_df[str(flank_length)].iloc[five_row]
                
        three_row = target2seq_occurence_max[fbtr][1] - 1 + flank_length
        if three_row < len(target2seq_occurence_max[fbtr][0]):
            if not np.isnan(plfold_df[str(flank_length)].iloc[three_row]):
                three_prime = plfold_df[str(flank_length)].iloc[three_row]
        
        if not np.isnan(five_prime) and not np.isnan(three_prime):
            flank_avg = (five_prime+three_prime)/2
            target2maxflank[fbtr] = [flank_avg]
    
    for fbtr, values in nontarget2seq_occurence_max.items():
        plfold_results = open(RNAplfold_direct + '\\' + fbtr + '_lunp', 'r')
        plfold_df = pd.read_csv(plfold_results, sep = '\t', skiprows = 1)
        plfold_df = plfold_df.rename(columns={' #i$': 'nt', 'l=1': '1'})
        
        five_row =  nontarget2seq_occurence_max[fbtr][1] - 1 - mer_length
        if five_row > 0:
            if not np.isnan(plfold_df[str(flank_length)].iloc[five_row]):
                five_prime = plfold_df[str(flank_length)].iloc[five_row]
                
        three_row = nontarget2seq_occurence_max[fbtr][1] - 1 + flank_length
        if three_row < len(nontarget2seq_occurence_max[fbtr][0]):
            if not np.isnan(plfold_df[str(flank_length)].iloc[three_row]):
                three_prime = plfold_df[str(flank_length)].iloc[three_row]
        
        if not np.isnan(five_prime) and not np.isnan(three_prime):
            flank_avg = (five_prime+three_prime)/2
            nontarget2maxflank[fbtr] = [flank_avg]
    
    
    num_significant = 0
    num_non_significant = 0
    p_value_list = []
    target_df = pd.DataFrame(target2maxflank).T
    nontarget_df = pd.DataFrame(nontarget2maxflank).T
    
    # for _ in range(num_replicates):
    target_nona = target_df.dropna(axis=0, subset=[0])
    nontarget_nona = nontarget_df.dropna(axis=0, subset=[0])
        # nontarget_nona_sample = nontarget_nona.sample(n = len(target_nona))
        
    stat, p_value = sp.stats.mannwhitneyu(target_nona[0], nontarget_nona[0])
    p_value_list.append(p_value)
        
    if p_value < 0.05:
        num_significant +=1
    else:
        num_non_significant +=1 
            
    print ('Number of significant runs = ' + str(num_significant))
    print ('Number of non-significant runs =  ' + str(num_non_significant))
    print (sum(p_value_list) / float(len(p_value_list)))
    
    return target_df, nontarget_df # for use in flank_access_histogram (df w/ structure: rows = fbtr and column = max_flank_value)

def flank_access_distribution(df, bin_increment, bin_max, percentage_tick, max_tick, RBP_name):
     #df returned in flank_max_wilcoxon
    #bin_increment is the increments on the y-axis, with bin_max being the max bin --> Judgement call based on data
    #percentage_ticks is the increment on x-axis inserted as a float (e.g. 2% increments = 0.02)
    # max_tick is the max y value --> judgement call; run with 1 first and then decide.
    
    #create bin for histogram
    n = 0
    bin_list = [n]
    while n < bin_max: 
        n += bin_increment
        bin_list.append(n)
    
    no_na = df.dropna()
    data = no_na[0]
    
    fig, ax = plt.subplots()
    y_vals, x_vals, e_ = ax.hist(data, bin_list , edgecolor='black')
    # y_max = round((max(y_vals) / len(data)) + 0.05, 2)
    y_max = max_tick + 0.05
    ax.set_yticks(ticks=np.arange(0.0, y_max * len(data), percentage_tick * len(data)))
    ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
    ax.set_xlabel('accessibility')
    ax.set_title(RBP_name)
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=len(data)))
    

In [ ]:
def motif_occurence_comparison(nested_dic):
    ''' nested_dic is a dic where key = motif and value = transcrip2seq_occurence_max 
    for which key = fbtr and values = ['seq', [occurence of motif w/ max access], [max access value]]
    Return dataframe that shows occurence of each variant of a motif in each transcript'''
    
    final_df = pd.DataFrame(nested_dic).applymap(lambda x: x[1])
        
    return final_df

# OUTPUTS:

In [ ]:
RBP_motif = 'CAWCWNCH'
motif_list = consensus_to_motifs(RBP_motif) 

pos_motifs = transcript_removal(target2seq, motif_list)
target_occurences = motif_occurences(pos_motifs, motif_list)
target_occurences_access = max_access(RNAplfold_dir, target_occurences, len(RBP_motif)) # Change mer_length

neg_motifs = transcript_removal(nontarget2seq, motif_list)
nontarget_occurences = motif_occurences(neg_motifs, motif_list)
nontarget_occurences_access = max_access(RNAplfold_dir, nontarget_occurences, len(RBP_motif)) #Change mer_length


In [ ]:
for motif in motif_list:
    print(motif)

In [ ]:
# flank accessibility output (once per flank length; change in third and fourth)
# flanking_access(transcript2seq_occurence_max, RNAplfold_direct, flank_length, mer_length)
for i in range(4,9):
    target_final_val = flanking_access(target_occurences_access, RNAplfold_dir, i, len(RBP_motif)) # Change flank_length and mer_length
    nontarget_final_val = flanking_access(nontarget_occurences_access, RNAplfold_dir, i, len(RBP_motif)) # Change flank_length and mer_length
    print (target_final_val, nontarget_final_val)

In [ ]:
# motif accessibilty 
targets_df, nontargets_df = motif_access_wilcoxon(target_occurences_access, nontarget_occurences_access) 

motif_access_distribution(targets_df, 0.1, 1, 0.05, 0.4, 'FMR1 Targets') #Change RBP name
motif_access_distribution(nontargets_df, 0.1, 1, 0.05, 0.4, 'FMR1 Nontargets') #Change RBP name

In [ ]:
# Flank access (once per flank length; change in first line)
# flank_access_wilcoxon(target2seq_occurence_max, nontarget2seq_occurence_max, RNAplfold_direct, flank_length, mer_length)
for i in range(4,9):
    targets_flank_df, nontargets_flank_df = flank_access_wilcoxon(target_occurences_access, nontarget_occurences_access, RNAplfold_dir, i, len(RBP_motif))
    flank_access_distribution(targets_flank_df, 0.1, 1, 0.05, 0.5, 'FMR1 Targets')
    flank_access_distribution(nontargets_flank_df, 0.1, 1, 0.05, 0.5, 'FMR1 Nontargets')

In [ ]:
out_dir_name = 'C:\\Users\\Alireza\\Documents\\MGY480\\seq_excerpt\\QKR58E1_nontargets.txt'
seq_excerpt(nontarget_occurences_access, out_dir_name)

In [ ]:
# SMG motif occurence analysis

smg_motifs = ('CNGG', 'CNGGN', 'CNGGNN', 'CNGGNNN', 'CNGGNNNN', 'CNGGNNNNN')
target_nest_dic = {}
nontarget_nest_dic = {}

for i in range(len(smg_motifs)):
    motif_list = consensus_to_motifs(smg_motifs[i]) # change motif

    pos_motifs = transcript_removal(target2seq, motif_list)
    target_occurences = motif_occurences(pos_motifs, motif_list)
    target_occurences_access = max_access(RNAplfold_dir, target_occurences, len(smg_motifs[i]))
    target_nest_dic[smg_motifs[i]] = target_occurences_access
    

    neg_motifs = transcript_removal(nontarget2seq, motif_list)
    nontarget_occurences = motif_occurences(neg_motifs, motif_list)
    nontarget_occurences_access = max_access(RNAplfold_dir, nontarget_occurences, len(smg_motifs[i]))
    nontarget_nest_dic[smg_motifs[i]] = nontarget_occurences_access

In [ ]:
target_smg = motif_occurence_comparison(target_nest_dic)
nontarget_smg = motif_occurence_comparison(nontarget_nest_dic)

In [ ]:
target_smg.to_excel('C:\\Users\\Alireza\\Documents\\MGY480\\smg_motif_variant_occurences\\targets.xlsx')
nontarget_smg.to_excel('C:\\Users\\Alireza\\Documents\\MGY480\\smg_motif_variant_occurences\\nontargets.xlsx')

In [ ]:
# For Logo generation
RBP_motif = 'UBCYB'
motif_list = consensus_to_motifs(RBP_motif) 
for motif in motif_list:
    print(motif)

In [ ]:
smg_targets_df = pd.read_excel('C:\\Users\\Alireza\\Documents\\MGY480\\smg_motif_variant_occurences\\targets.xlsx')
smg_nontargets_df = pd.read_excel('C:\\Users\\Alireza\\Documents\\MGY480\\smg_motif_variant_occurences\\nontargets.xlsx')

In [ ]:
smg_targets_df_new = pd.DataFrame()
smg_targets_df_new["CNGG"] = smg_targets_df["CNGG"] 
smg_targets_df_new["CNGGN"] = smg_targets_df["CNGGN"] - 1
smg_targets_df_new["CNGGNN"] = smg_targets_df["CNGGNN"] - 2
smg_targets_df_new["CNGGNNN"] = smg_targets_df["CNGGNNN"] - 3
smg_targets_df_new["CNGGNNNN"] = smg_targets_df["CNGGNNNN"] - 4
smg_targets_df_new["CNGGNNNNN"] = smg_targets_df["CNGGNNNNN"] - 5
smg_targets_df_new['Score'] = smg_targets_df_new.apply(lambda row: row.value_counts().iloc[0], axis = 1)

smg_nontargets_df_new = pd.DataFrame()
smg_nontargets_df_new["CNGG"] = smg_nontargets_df["CNGG"] 
smg_nontargets_df_new["CNGGN"] = smg_nontargets_df["CNGGN"] - 1
smg_nontargets_df_new["CNGGNN"] = smg_nontargets_df["CNGGNN"] - 2
smg_nontargets_df_new["CNGGNNN"] = smg_nontargets_df["CNGGNNN"] - 3
smg_nontargets_df_new["CNGGNNNN"] = smg_nontargets_df["CNGGNNNN"] - 4
smg_nontargets_df_new["CNGGNNNNN"] = smg_nontargets_df["CNGGNNNNN"] - 5
smg_nontargets_df_new['Score'] = smg_nontargets_df_new.apply(lambda row: row.value_counts().iloc[0], axis = 1)

In [ ]:
bin_list = [1,2,3,4,5,6,7]
data = smg_targets_df_new['Score']

fig, ax = plt.subplots()
y_vals, x_vals, e_ = ax.hist(data, bin_list , edgecolor='black')
y_max = 0.35 + 0.05
ax.set_yticks(ticks=np.arange(0.0, y_max * len(data), 0.05 * len(data)))
ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
ax.set_xlabel('Score')
ax.set_title('SMG Targets')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=len(data)))

In [ ]:
bin_list = [1,2,3,4,5,6,7]
data = smg_nontargets_df_new['Score']

fig, ax = plt.subplots()
y_vals, x_vals, e_ = ax.hist(data, bin_list , edgecolor='black')
y_max = 0.35 + 0.05
ax.set_yticks(ticks=np.arange(0.0, y_max * len(data), 0.05 * len(data)))
ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
ax.set_xlabel('Score')
ax.set_title('SMG Nontargets')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=len(data)))